# Example of resolution of a Non Linear Mixed Effect Model
In this example, we have a ODE with several parameters, and we used First Order Conditional Estimation (FOCE) method to find the parameters values.

The individual datas are artificially generated using the actual parameter values and by adding some variability and noise. 

In NonMem, parameters for the i-th individual are written with the following form:

$$K_i = K_{pop} e^{\eta_i}, \eta_i \sim \ N(0,\omega_{pop})$$

where $K_{pop}$ and $\omega_{pop}$ are population specific. 


The goal of this cookbook is to find the following values:
- At the population level: $K_{pop}$ and $\omega_{pop}$
- At the individual level: $\eta_i$

And checks that the $\eta_i \sim \ N(0,\omega_{pop})$

In [ ]:
# All imports
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.integrate import solve_ivp
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

## Problem definitions

### Definition of the ODE

In [ ]:
# Define the ODE function
def ode_model(t, C, K):
    return - K * C

y0 = [100]  # Initial ODE condition

outputName = "Concentration"

# Define the population level parameters
K_pop = 3  # Typical elimination rate constant
Var_pop = 0.5  # Between-subject variability (log-normal)
sigma_noise = 0.2  # Residual error


# Define the initial conditions for the optmization algorithm (close the aimed value)
K_pop_init = 5 # population-level parameter
Var_pop_init = 1  # Random effect
# Combine them into one initial parameter vector
initial_params = np.hstack([K_pop_init, Var_pop_init])
bounds = [(0.01, None)] * len(initial_params)  # to avoid unrealistic values of K that can make the model go crazy.


### Creation of the dataset

In [ ]:
# Generate synthetic data for 5 individuals, and 10 observations
np.random.seed(43)
n_individuals = 5
n_obs = 10
t_max = 4
time_points = np.linspace(0, t_max, n_obs)  # 10 time points from 0 to 10

# Simulated dataset
data = []

for i in range(n_individuals):
    eta_pop = np.random.normal(0, Var_pop)  # Random effect
    Kel_i = np.multiply(K_pop, np.exp(eta_pop))  # Individual parameters

    # Solve the ODE
    sol = solve_ivp(ode_model, [0, max(time_points)], y0, t_eval=time_points, args=(Kel_i,))
    # Add measurement noise
    noise = np.random.normal(0, sigma_noise, size=n_obs)
    C_obs = sol.y[0] + noise

    # Store data
    for t, C in zip(time_points, C_obs):
        data.append([i, t, C])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Individual", "Time", outputName])

# Select the first 3 patients at most to check
individual_ids = df['Individual'].unique()[:min(3, n_individuals )]

plt.figure(figsize=(8, 6))

for individual_id in individual_ids:
    individual_data = df[df['Individual'] == individual_id]
    plt.plot(individual_data['Time'], individual_data[outputName], label=f'Individual {individual_id}', marker='o')

plt.xlabel('Time')
plt.ylabel(outputName)
plt.title(f'Evolution of {outputName} for each Individual')
plt.legend(title='Individuals')
plt.grid(True)
plt.show()

## Optimization functions definitions
We defined the FOCE likelihood function (and a tool function for readibility)

In [ ]:
# Function to simulate concentration given K
def simulate_model(K, y0, time_points):
    sol = solve_ivp(ode_model, [0, max(time_points)], y0, t_eval=time_points, args=(K,))
    return sol.y[0]

def obj_fun_eta_star(eta_i, parameters, C_obs, time_points, sigma_noise):
    theta = parameters[0]
    omega = parameters[1]

    K_i = theta * np.exp(eta_i)

    C_pred       = simulate_model(K_i, y0, time_points)
    residuals = C_obs - C_pred

    return np.sum( residuals**2 / (2 * sigma_noise**2)) + eta_i**2 / (2 * omega**2)



# FOCE likelihood function
def foce_minus_log_likelihood(params, df, n_individuals, time_points, sigma_noise):
    theta = params[0]  # Population parameter
    omega = params[1]  # Variability 

    log_likelihood = 0
    for i in range(n_individuals):
        # Extract observed data for this individual
        C_obs = df[df["Individual"] == i][outputName].values

        # Find eta_i_star
        eta_i=0
        opt_result = minimize(obj_fun_eta_star, eta_i, args=(params, C_obs, time_points, sigma_noise))
        eta_i_star = opt_result.x[0]

        # Establish individual parameter value
        K_i = theta * np.exp(eta_i_star)
        # Predicted values for C, C' and C''
        C_pred       = simulate_model(K_i, y0, time_points)
        C_dt_pred    = ode_model(t, C_pred, K_i)
        C_dt_dt_pred = ode_model(t, C_dt_pred, K_i)

        # Compute residuals and intermediate outputs
        #print(C_pred)
        residuals = C_obs - C_pred
        derivative_squared = C_dt_pred**2
        taylor_expansion = residuals * C_dt_dt_pred 

        # Compute likelihood 
        ## Compute likelihood of the main part at the individual level
        a = np.sum((residuals / sigma_noise) ** 2 + np.log(2 * np.pi * sigma_noise**2))

        ## Compute likelihood of the population level
        b = np.log(2 * np.pi * omega) + (eta_i/omega)**2

        ## Compute likelihood of the taylor expansion part at the individual level
        c = - np.log(2 * np.pi) + np.log(abs(-1/(omega**2) + np.sum( (- derivative_squared + taylor_expansion )/(sigma_noise**2))))

        
        log_likelihood += a + b + c

    print(f"For theta = {theta:.2e} and omega = {omega:.2e}, We have L = {log_likelihood:.3e}")
    return (- log_likelihood)  # Minimize negative log-likelihood


log_likelihood = foce_minus_log_likelihood([5.675439391580754,0.26258449412761864], df, n_individuals, time_points, sigma_noise)

## Optimization algorithm and visualization of the results

### Optimization

In [ ]:
# Combine them into one initial parameter vector
initial_params = np.hstack([K_pop_init, Var_pop_init])
bounds = [(0.01, None)] * len(initial_params) 

# Optimization
opt_result = minimize(foce_minus_log_likelihood, initial_params, args=(df, n_individuals, time_points, sigma_noise),
                      bounds = bounds, method='Nelder-Mead', options={"maxiter":10})  

# Extract results
K_pop_opt = opt_result.x[0]
Var_pop_opt = opt_result.x[1]
params_opt = [K_pop_opt, Var_pop_opt]

print(f"Maximum L: {-opt_result.fun:.3e}")
print(f"Optimized K_pop: {K_pop_opt}")
print(f"Optimized Var_pop: {Var_pop_opt}")

### Plot the results
We compare the optimization outputs with the dataset

In [ ]:
# Function to plot observed vs. predicted concentrations
#K_pop_opt = 2
#Var_pop_opt = 0.5

def plot_results(df, theta_kel_opt, time_points):
    plt.figure(figsize=(10, 6))

    for i in range(n_individuals):  # Loop over individuals
        C_obs = df[df["Individual"] == i][outputName].values

        # Find eta_i_star
        eta_i=0
        opt_result = minimize(obj_fun_eta_star, eta_i, args=(params_opt, C_obs, time_points, sigma_noise))
        eta_i_star = opt_result.x[0]
        print(f"i = {i}: exp(eta_i*) = {np.exp(eta_i_star):.2e}")
        Kel_i = theta_kel_opt * np.exp(eta_i_star)  # Individual-specific Kel
        C_pred = simulate_model(Kel_i, y0, time_points)  # Model predictions

        # Extract observed data for individual i
        df_i = df[df["Individual"] == i]

        # Scatter plot for observed data
        plt.scatter(df_i["Time"], df_i[outputName], label=f"Observed - ID {i}")

        # Line plot for model-predicted concentrations
        plt.plot(time_points, C_pred, linestyle='--', label=f"Predicted - ID {i} - eta_i={eta_i_star:.2e}")

    plt.xlabel("Time")
    plt.ylabel(outputName)
    plt.title("Observed vs. FOCE Model-Predicted Output")
    plt.legend()
    plt.grid()
    plt.show()

# Call the function to plot results
plot_results(df, K_pop_opt, time_points)


### Diagnostics

In [ ]:

# Compute residuals and metrics
def compute_diagnostics(df, theta_kel_opt, time_points):
    residuals = []
    all_obs = []
    all_pred = []

    for i in range(5):
        C_obs = df[df["Individual"] == i][outputName].values

        # Find eta_i_star
        eta_i=0
        opt_result = minimize(obj_fun_eta_star, eta_i, args=(params_opt, C_obs, time_points, sigma_noise))
        eta_i_star = opt_result.x[0]
        Kel_i = theta_kel_opt * np.exp(eta_i_star)
        C_pred = simulate_model(Kel_i, y0, time_points)
        
        residuals.extend(C_obs - C_pred)
        all_obs.extend(C_obs)
        all_pred.extend(C_pred)

    # Compute metrics
    mse = mean_squared_error(all_obs, all_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(all_obs, all_pred)

    print(f"MSE (Lower values indicate a better fit) : {mse:.4f}")
    print(f"RMSE (Lower values indicate a better fit): {rmse:.4f}")
    print(f"R² (Close to 1 means the model explains most variability): {r2:.4f}")

    return residuals, all_obs, all_pred

# Run diagnostics
residuals, all_obs, all_pred = compute_diagnostics(df, K_pop_opt, time_points)

# Plot Residuals
plt.figure(figsize=(10, 5))
plt.scatter(all_obs, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Observed outputs")
plt.ylabel("Residuals")
plt.title("Residual Plot (Ideally, residuals should be randomly scattered around zero) ")
plt.grid()
plt.show()

# QQ-Plot of Residuals
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("QQ-Plot of Residuals (Helps check if residuals follow a normal distribution)")
plt.show()


In [ ]:
K=2
C_pred       = simulate_model(K, y0, time_points)
C_dt_pred    = ode_model(t, C_pred, K)
C_dt_dt_pred = ode_model(t, C_dt_pred, K)

residuals = C_obs - C_pred
derivative_squared = C_dt_pred**2
taylor_expansion = residuals * C_dt_dt_pred 

print(residuals)
print(derivative_squared)
print(taylor_expansion)